In [2]:
import json
import pandas as pd

from pyspark.sql import SparkSession

In [22]:
metadata_file = "10000_metadata.json"
tracking_file = "10000_tracking.txt"

In [175]:
with open(metadata_file, "r", encoding='UTF-8') as f:
    metadata_str = f.read()

metadata = json.loads(metadata_str)

In [5]:
metadata_game_id = metadata["id"]

# Metadata File Exploration #

In [92]:
from pprint import pprint

pprint(metadata)

{'away_team': {'acronym': 'SHE',
               'id': 746,
               'name': 'Sheffield United',
               'short_name': 'Sheffield'},
 'away_team_coach': {'first_name': 'Chris', 'id': 570, 'last_name': 'Wilder'},
 'away_team_kit': {'id': 2661,
                   'jersey_color': '#267731',
                   'name': 'away',
                   'number_color': '#bebd4a',
                   'season': {'end_year': 2021,
                              'id': 7,
                              'name': '2020/2021',
                              'start_year': 2020},
                   'team_id': 746},
 'away_team_score': 1,
 'ball': {'trackable_object': 55},
 'competition_edition': {'competition': {'area': 'ENG',
                                         'id': 1,
                                         'name': 'Premier League'},
                         'id': 147,
                         'name': 'ENG - Premier League - 2020/2021',
                         'season': {'end_year': 2021,
  

In [9]:
len(metadata.keys())

20

In [10]:
metadata.keys()

dict_keys(['id', 'home_team_score', 'away_team_score', 'date_time', 'stadium', 'home_team', 'home_team_kit', 'away_team', 'away_team_kit', 'home_team_coach', 'away_team_coach', 'competition_edition', 'competition_round', 'referees', 'players', 'status', 'home_team_side', 'ball', 'pitch_length', 'pitch_width'])

In [17]:
metadata_game_id

10000

The metadata looks straight-forward. The entire metadata is to inform a match, which includes multiple entities, including __Teams__, __Players__, __Referees__, __Competition__.

In [34]:
metadata_df = pd.DataFrame.from_dict(metadata, orient='index')
metadata_df = metadata_df.transpose()
metadata_df.shape

# type(metadata)

(1, 20)

In [85]:
type(metadata)

dict

In [176]:
players_df = pd.DataFrame.from_dict(metadata.pop('players'))

players_df.shape

(36, 18)

In [178]:
players_df.dtypes

player_role         object
start_time          object
end_time            object
number               int64
yellow_card          int64
red_card             int64
injured               bool
goal                 int64
own_goal             int64
team_player_id       int64
team_id              int64
id                   int64
first_name          object
last_name           object
short_name          object
birthday            object
trackable_object     int64
gender              object
dtype: object

In [90]:
len(metadata.keys())

19

## Tracking Exploration ##

In [6]:
f = open(tracking_file, "r")
lines = f.readlines()

tracking = []
for line in lines:
    tracking.append(json.loads(line))

In [7]:
tracking_game_id = tracking_file.split("_")[0]

In [8]:
len(tracking)

57748

In [9]:
tracking[0:5]

[{'data': [],
  'possession': {'group': None, 'trackable_object': None},
  'frame': 0,
  'image_corners_projection': [],
  'timestamp': None,
  'period': None},
 {'data': [],
  'possession': {'group': None, 'trackable_object': None},
  'frame': 1,
  'image_corners_projection': [],
  'timestamp': None,
  'period': None},
 {'data': [],
  'possession': {'group': None, 'trackable_object': None},
  'frame': 2,
  'image_corners_projection': [],
  'timestamp': None,
  'period': None},
 {'data': [],
  'possession': {'group': None, 'trackable_object': None},
  'frame': 3,
  'image_corners_projection': [],
  'timestamp': None,
  'period': None},
 {'data': [],
  'possession': {'group': None, 'trackable_object': None},
  'frame': 4,
  'image_corners_projection': [],
  'timestamp': None,
  'period': None}]

In [10]:
tracking[57747]

{'data': [{'track_id': 11565,
   'trackable_object': 11565,
   'is_visible': False,
   'x': -30.79,
   'y': -3.37},
  {'track_id': 3267,
   'trackable_object': 3267,
   'is_visible': False,
   'x': -0.78,
   'y': -24.53},
  {'track_id': 11537,
   'trackable_object': 11537,
   'is_visible': False,
   'x': -1.79,
   'y': -12.67},
  {'track_id': 11847,
   'trackable_object': 11847,
   'is_visible': True,
   'x': 7.91,
   'y': -30.35},
  {'track_id': 851,
   'trackable_object': 851,
   'is_visible': False,
   'x': 3.3,
   'y': 3.99},
  {'track_id': 3847,
   'trackable_object': 3847,
   'is_visible': True,
   'x': 24.22,
   'y': -30.23},
  {'track_id': 5318,
   'trackable_object': 5318,
   'is_visible': True,
   'x': 15.02,
   'y': -22.19},
  {'track_id': 11879,
   'trackable_object': 11879,
   'is_visible': True,
   'x': 24.36,
   'y': -26.31},
  {'track_id': 4683,
   'trackable_object': 4683,
   'is_visible': True,
   'x': 13.29,
   'y': -12.16},
  {'track_id': 9449,
   'trackable_object'

A lot of questions here, including how does the tracking data relate to the match metadata? What is the list of double in _image_corners_projection_ about?

_trackable_object_ is actually in both sets

In [29]:
metadata["players"][2]

{'player_role': {'id': 15, 'name': 'Center Forward', 'acronym': 'CF'},
 'start_time': '01:22:54',
 'end_time': None,
 'number': 18,
 'yellow_card': 0,
 'red_card': 0,
 'injured': False,
 'goal': 0,
 'own_goal': 0,
 'team_player_id': 23626,
 'team_id': 2,
 'id': 11773,
 'first_name': 'Takumi',
 'last_name': 'Minamino',
 'short_name': 'T. Minamino',
 'birthday': '1995-01-16',
 'trackable_object': 11879,
 'gender': 'male'}

In [46]:
for player in metadata["players"]:
    if player['trackable_object'] == 851:
        pprint(player)

{'birthday': '1994-03-11',
 'end_time': None,
 'first_name': 'Andrew',
 'gender': 'male',
 'goal': 0,
 'id': 841,
 'injured': False,
 'last_name': 'Robertson',
 'number': 26,
 'own_goal': 0,
 'player_role': {'acronym': 'LWB', 'id': 5, 'name': 'Left Wing Back'},
 'red_card': 0,
 'short_name': 'A. Robertson',
 'start_time': '00:00:00',
 'team_id': 2,
 'team_player_id': 5532,
 'trackable_object': 851,
 'yellow_card': 0}


In [23]:
len(tracking[1000]['data'])

23

There are 23 trackable objects. What else is _trackable_object_ beside the 22 players from both teams?

Apparently, the ball (_trackable_object_ = 55) is also tracked.

In [24]:
tracking[1000].keys()

dict_keys(['data', 'possession', 'frame', 'image_corners_projection', 'timestamp', 'period'])

# Extract

In [2]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

23/12/04 17:19:54 WARN Utils: Your hostname, ALDO-DESKTOP resolves to a loopback address: 127.0.1.1; using 172.19.19.57 instead (on interface eth0)
23/12/04 17:19:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/04 17:19:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pyspark.pandas as ps

/home/reyadji/.pyenv/versions/3.10.12/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


### Using Spark

In [71]:
spark_tracking_df = spark.read.text(tracking_file, lineSep="\n")

In [72]:
spark_tracking_df.count()

57748

In [73]:
spark_tracking_df.printSchema()

root
 |-- value: string (nullable = true)



In [74]:
spark_tracking_df.show()

+--------------------+
|               value|
+--------------------+
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [], "pos...|
|{"data": [{"track...|
|{"data": [{"track...|
|{"data": [{"track...|
|{"data": [{"track...|
|{"data": [{"track...|
|{"data": [{"track...|
|{"data": [{"track...|
|{"data": [{"track...|
|{"data": [{"track...|
|{"data": [{"track...|
+--------------------+
only showing top 20 rows



The most important data (track_id/trackable_object) is in a list under data, which is a list itself for the entire match, e.g. tracking[{'data':[{'track_id',....}]}, ... ]

The dataframe needs to be flatten until the track_id is at the top level of the dataframe.

In [37]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *

In [ ]:
# tracking_schema = ArrayType(
#     StructType([StructField("data", ArrayType(

# df2 = df.withColumn("attribute3", from_json(df.attribute3, schema))
spark_tracking_df = 

### Using pandas

In [11]:
tracking_game_id = tracking_file.split("_")[0]

tracking_game_id

'10000'

In [52]:
pandas_tracking_df = pd.DataFrame(tracking)
pandas_tracking_df["game_id"] = tracking_game_id

pandas_tracking_df.sample(3)

,data,possession,frame,image_corners_projection,timestamp,period,game_id
23675,"[{'track_id': 512, 'trackable_object': 512, 'i...","{'group': None, 'trackable_object': None}",23675,[],00:39:26.50,1.0,10009
2167,"[{'track_id': 512, 'trackable_object': 512, 'i...","{'group': 'away team', 'trackable_object': None}",2167,"[-50.266857708833854, 39.0, -18.04812532113806...",00:03:35.70,1.0,10009
11029,[],"{'group': None, 'trackable_object': None}",11029,[],00:18:21.90,1.0,10009


In [15]:
pandas_tracking_df.shape

(57748, 7)

Filter rows where there is empty data

In [50]:
# Filter rows where there is no tracking data
pandas_tracking_df = pandas_tracking_df[pandas_tracking_df["data"].apply(lambda x: len(x)) > 0]

pandas_tracking_df.shape

(43503, 6)

In [19]:
pandas_tracking_df.head(5)

,data,possession,frame,image_corners_projection,timestamp,period,game_id
10,"[{'track_id': 11565, 'trackable_object': 11565...","{'group': None, 'trackable_object': None}",10,[],00:00:00.00,1.0,10000
11,"[{'track_id': 11565, 'trackable_object': 11565...","{'group': None, 'trackable_object': None}",11,[],00:00:00.10,1.0,10000
12,"[{'track_id': 11565, 'trackable_object': 11565...","{'group': None, 'trackable_object': None}",12,[],00:00:00.20,1.0,10000
13,"[{'track_id': 11565, 'trackable_object': 11565...","{'group': None, 'trackable_object': None}",13,[],00:00:00.30,1.0,10000
14,"[{'track_id': 11565, 'trackable_object': 11565...","{'group': None, 'trackable_object': None}",14,[],00:00:00.40,1.0,10000


Split the dataframe into 3 different entities which might translate into three SQL tables: Frame, Posession, Track

In [27]:
track_df = pandas_tracking_df[["game_id", "data", "frame"]]

track_df.sample(2)

,game_id,data,frame
29240,10000,"[{'track_id': 11565, 'trackable_object': 11565...",29240
22011,10000,"[{'track_id': 11565, 'trackable_object': 11565...",22011


In [28]:
track_df.shape

(45697, 3)

In [67]:
possession_df = pandas_tracking_df[["game_id", "possession"]]
possession_df = possession_df.join(pd.json_normalize(possession_df.pop("possession")))
possession_df['trackable_object'] = possession_df['trackable_object'].astype('Int64').astype(str)
possession_df.sample(10)

,game_id,group,trackable_object
928,10009,None,<NA>
54103,10009,away team,<NA>
41709,10009,home team,<NA>
10128,10009,home team,8259
26511,10009,home team,<NA>
26120,10009,away team,<NA>
21014,10009,None,<NA>
43618,10009,None,<NA>
27851,10009,home team,<NA>
51910,10009,None,<NA>


In [68]:
possession_df.dtypes

game_id             object
group               object
trackable_object    object
dtype: object

In [30]:
possession_df.shape

(45697, 2)

In [164]:
frame_df = pandas_tracking_df[["game_id", "frame", "image_corners_projection", "timestamp", "period"]]
# df['time'] = pd.to_datetime(df['time'])
# df['time_in_seconds'] = df['time'].dt.hour * 3600 + df['time'].dt.minute * 60 + df['time'].dt.second + df['time'].dt.microsecond / 1e6
frame_df = frame_df.dropna(subset=['timestamp'])
frame_df['timestamp'] = pd.to_datetime(frame_df['timestamp'])
frame_df['timestamp_in_seconds'] = frame_df['timestamp'].dt.hour * 3600 + frame_df['timestamp'].dt.minute * 60 + frame_df['timestamp'].dt.second
frame_df['timestamp_in_seconds'] = frame_df['timestamp_in_seconds'].astype(int)
frame_df = frame_df.drop(['timestamp'], axis=1)
frame_df["period"] = frame_df["period"].astype(int)

frame_df.sample(2)

/tmp/ipykernel_19981/447665661.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  frame_df['timestamp'] = pd.to_datetime(frame_df['timestamp'])


,game_id,frame,image_corners_projection,period,timestamp_in_seconds
25322,10009,25322,"[39.62581180168092, 39.0, 19.646419067677787, ...",1,2531
43121,10009,43121,"[-21.894645863958505, 39.0, -10.25114134262234...",2,4236


In [156]:
frame_df[frame_df['timestamp_in_seconds'].isna()]

,game_id,frame,image_corners_projection,timestamp,period,timestamp_in_seconds
0,10009,0,[],NaT,NaN,NaN
1,10009,1,[],NaT,NaN,NaN
2,10009,2,[],NaT,NaN,NaN
3,10009,3,[],NaT,NaN,NaN
4,10009,4,[],NaT,NaN,NaN
5,10009,5,[],NaT,NaN,NaN
6,10009,6,[],NaT,NaN,NaN
7,10009,7,[],NaT,NaN,NaN
8,10009,8,[],NaT,NaN,NaN
9,10009,9,[],NaT,NaN,NaN


In [168]:
frame_df.sample(2)

,game_id,frame,image_corners_projection,period,timestamp_in_seconds
55680,10009,55680,[],2,5492
26036,10009,26036,"[-47.1391293561168, 38.99999999999999, -21.320...",1,2602


In [71]:
frame_df.shape

(58140, 5)

In [142]:
frame_df.dtypes

game_id                      object
frame                         int64
image_corners_projection     object
timestamp                    object
period                      float64
dtype: object

#### Explode/flatten all nested data in Track and Possesion DF

In [33]:
track_df = track_df.explode('data').reset_index(drop=True)

track_df.sample(2)

,game_id,data,frame
936611,10000,"{'track_id': 3847, 'trackable_object': 3847, '...",51756
756570,10000,"{'track_id': 2776, 'trackable_object': 2776, '...",42680


In [34]:
track_df.shape

(1051031, 3)

In [35]:
track_df = track_df.join(pd.json_normalize(track_df.pop("data")))

# df = df.join(pd.json_normalize(df.pop('item_order')))

track_df.sample(2)

,game_id,frame,track_id,trackable_object,is_visible,x,y,z
827075,10000,45856,18726,18726,True,-1.97,-18.53,NaN
13371,10000,591,2776,2776,False,0.14,26.01,NaN


In [36]:
track_df.shape

(1051031, 8)

In [61]:
track_df.dtypes

game_id              object
frame                 int64
track_id              int64
trackable_object      int64
is_visible             bool
x                   float64
y                   float64
z                   float64
dtype: object

# Load

## Parquet

Load dataframe into parquet files.

In [182]:
track_df.to_parquet("track.parquet", engine="fastparquet", append=True)

FileNotFoundError: track.parquet

Extract the data back to pandas dataframe for quick sanity check

In [41]:
test_track_df = pd.read_parquet("track.parquet")

test_track_df.shape

(1051031, 8)

In [42]:
test_track_df.head(2)

,game_id,frame,track_id,trackable_object,is_visible,x,y,z
0,10000,10,11565,11565,False,39.02,-1.15,NaN
1,10000,10,3267,3267,False,19.51,-4.78,NaN


Append the next batch of tracking dataset into the same parquet file

In [3]:
def transform_track(input_df):
    track_df = input_df[["game_id", "data", "frame"]]
    track_df = track_df.explode('data').reset_index(drop=True)
    track_df = track_df.join(pd.json_normalize(track_df.pop("data")))
    return track_df

In [6]:
tracking_file = "10009_tracking.txt"

f = open(tracking_file, "r")
lines = f.readlines()

tracking = []
for line in lines:
    tracking.append(json.loads(line))

tracking_game_id = tracking_file.split("_")[0]

pandas_tracking_df = pandas_tracking_df[pandas_tracking_df["data"].apply(lambda x: len(x)) > 0]

track_df = transform_track(pandas_tracking_df)
track_df.to_parquet("track.parquet", engine="fastparquet", append=True)

In [74]:
track_df.dtypes

game_id              object
frame                 int64
track_id              int64
trackable_object      int64
is_visible             bool
x                   float64
y                   float64
z                   float64
dtype: object

In [75]:
track_df['trackable_object'] = track_df['trackable_object'].astype(str)

In [78]:
track_df.loc[199654]['trackable_object'] == '18776'

True

In [185]:
test_track_df = pd.read_parquet("track.parquet")

test_track_df.shape

(0, 0)

In [9]:
test_track_df["game_id"].unique()

array(['10000', '10009'], dtype=object)

In [13]:
pandas_tracking_df = pd.read_json(tracking_file, lines=True)

pandas_tracking_df.shape

(58140, 6)

In [15]:
pandas_tracking_df.head(2)

,data,possession,frame,image_corners_projection,timestamp,period
0,[],"{'group': None, 'trackable_object': None}",0,[],NaT,NaN
1,[],"{'group': None, 'trackable_object': None}",1,[],NaT,NaN


## SQL

Load dataframe into local sqlite3 DB. First, establish a connection

In [166]:
import sqlite3

con = sqlite3.connect('srcftbl')

In [41]:
track_df.to_sql(name="track", con=con, if_exists='append')

990620

In [43]:
test_df = pd.read_sql('select * from track limit 2', con)

In [44]:
test_df.shape

(2, 9)

In [45]:
test_df

,index,game_id,frame,track_id,trackable_object,is_visible,x,y,z
0,0,10009,10,512,512,0,40.24,-1.65,None
1,1,10009,10,4713,4713,0,17.17,-4.61,None


In [47]:
track_df.head(2)

,game_id,frame,track_id,trackable_object,is_visible,x,y,z
0,10009,10,512,512,False,40.24,-1.65,NaN
1,10009,10,4713,4713,False,17.17,-4.61,NaN


In [93]:
len(frame_df["frame"].unique())

58140

In [94]:
frame_df.columns

Index(['game_id', 'frame', 'image_corners_projection', 'timestamp', 'period'], dtype='object')

In [96]:
frame_df["timestamp"].max() - frame_df["timestamp"].min()

Timedelta('0 days 01:35:37.900000')

Testing all the transformation

In [186]:
con = sqlite3.connect('srcftbl.db')

In [188]:
test_df = pd.read_sql('select * from track limit 2', con)

test_df

,index


In [105]:
metadata_df = pd.json_normalize(metadata)


In [110]:
metadata_df

,id,home_team_score,away_team_score,date_time,referees,players,status,home_team_side,pitch_length,pitch_width,...,competition_edition.season.id,competition_edition.season.start_year,competition_edition.season.end_year,competition_edition.season.name,competition_edition.name,competition_round.id,competition_round.name,competition_round.round_number,competition_round.potential_overtime,ball.trackable_object
0,10000,2,1,2020-10-24T19:00:00Z,[],"[{'player_role': {'id': 9, 'name': 'Left Midfi...",closed,"[right_to_left, left_to_right]",101,68,...,7,2020,2021,2020/2021,ENG - Premier League - 2020/2021,12,Round 6,6,False,55


In [112]:
metadata_df = metadata_df.map(str)

In [113]:
metadata_df.dtypes

id                                       object
home_team_score                          object
away_team_score                          object
date_time                                object
referees                                 object
players                                  object
status                                   object
home_team_side                           object
pitch_length                             object
pitch_width                              object
stadium.id                               object
stadium.name                             object
stadium.city                             object
stadium.capacity                         object
home_team.id                             object
home_team.name                           object
home_team.short_name                     object
home_team.acronym                        object
home_team_kit.id                         object
home_team_kit.team_id                    object
home_team_kit.season.id                 

In [189]:
metadata_df

,id,home_team_score,away_team_score,date_time,referees,players,status,home_team_side,pitch_length,pitch_width,...,competition_edition.season.id,competition_edition.season.start_year,competition_edition.season.end_year,competition_edition.season.name,competition_edition.name,competition_round.id,competition_round.name,competition_round.round_number,competition_round.potential_overtime,ball.trackable_object
0,10000,2,1,2020-10-24T19:00:00Z,[],"[{'player_role': {'id': 9, 'name': 'Left Midfi...",closed,"['right_to_left', 'left_to_right']",101,68,...,7,2020,2021,2020/2021,ENG - Premier League - 2020/2021,12,Round 6,6,False,55


In [119]:
players_df.head(1)

,player_role,start_time,end_time,number,yellow_card,red_card,injured,goal,own_goal,team_player_id,team_id,id,first_name,last_name,short_name,birthday,trackable_object,gender
0,"{'id': 9, 'name': 'Left Midfield', 'acronym': ...",01:15:33,None,17,0,0,False,0,0,15719,746,18534,David,McGoldrick,D. McGoldrick,1987-11-29,18731,male


In [179]:
players_df.dtypes

player_role         object
start_time          object
end_time            object
number               int64
yellow_card          int64
red_card             int64
injured               bool
goal                 int64
own_goal             int64
team_player_id       int64
team_id              int64
id                   int64
first_name          object
last_name           object
short_name          object
birthday            object
trackable_object     int64
gender              object
dtype: object

In [180]:
players_df['injured'] = players_df['injured'].astype(int)
players_df['player_role'] = players_df['player_role'].astype(str)

In [181]:
players_df.to_sql(name="player", con=con, if_exists='append')

36

In [123]:
track_df.dtypes

game_id              object
frame                 int64
track_id              int64
trackable_object     object
is_visible             bool
x                   float64
y                   float64
z                   float64
dtype: object

In [126]:
track_df.to_sql(name="track", con=con, if_exists='append')

990620

In [124]:
possession_df.dtypes

game_id             object
group               object
trackable_object    object
dtype: object

In [127]:
possession_df.to_sql(name="possession", con=con, if_exists='append')

58140

In [165]:
frame_df.dtypes

game_id                     object
frame                        int64
image_corners_projection    object
period                       int64
timestamp_in_seconds         int64
dtype: object

In [169]:
frame_df['image_corners_projection'] = frame_df['image_corners_projection'].astype(str) 

In [170]:
rows = frame_df.to_sql(name="frame", con=con, if_exists='append')

58121

In [184]:
srfctbl_db = "srcftbl.db"
import os
os.path.isexist(srfctbk_df)

AttributeError: module 'posixpath' has no attribute 'isexist'